In [1]:
%load_ext autoreload
%autoreload 2

# For Stack

Generate mock responses

In [2]:
from s1_frame_enumerator import S1Frame, query_slc_metadata_over_frame

In [3]:
from s1_frame_enumerator import S1Frame, query_slc_metadata_over_frame
import json

In [4]:
def generate_mock_data(frame_id):
    frame = S1Frame(frame_id)
    result = query_slc_metadata_over_frame(frame)
    out_path = f'frame_{frame_id}_asf_results.json'
    json.dump(result, open(out_path, 'w'))
    return out_path

In [5]:
# generate_mock_data(13403)
# generate_mock_data(13404)
# generate_mock_data(9847)
# generate_mock_data(9848)

'frame_9848_asf_results.json'

# For Enumeration

Generate Small Stack

In [6]:
from shapely.geometry import Point
import geopandas as gpd
from rasterio.crs import CRS
from s1_frame_enumerator import get_overlapping_s1_frames, get_s1_stack
import pandas as pd

aoi_geo = Point(-120, 35).buffer(.5)
df_aoi = gpd.GeoDataFrame(geometry=[aoi_geo], crs=CRS.from_epsg(4326))

In [7]:
frames = get_overlapping_s1_frames(aoi_geo, track_numbers=[137])
len(frames)

2

In [8]:
for frame in frames:
    print(frame.frame_id)

21248
21249


In [10]:
df_stack = get_s1_stack(frames)
df_stack.head()

,slc_id,start_time,stop_time,url,track_number,polarization,orbit,beam_mode,size_gb,stack_repeat_pass_id,repeat_pass_timestamp,geometry
0,S1A_IW_SLC__1SDV_20150401T015800_20150401T0158...,2015-04-01 01:58:00+00:00,2015-04-01 01:58:30+00:00,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,137,VV+VH,5284,IW,4.907014,3,2015-04-01 00:00:00+00:00,"POLYGON ((-121.18324 35.18838, -120.76283 33.3..."
1,S1A_IW_SLC__1SDV_20150401T015828_20150401T0158...,2015-04-01 01:58:28+00:00,2015-04-01 01:58:55+00:00,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,137,VV+VH,5284,IW,4.697075,4,2015-04-01 00:00:00+00:00,"POLYGON ((-121.54370 36.73265, -121.14296 35.0..."
2,S1A_IW_SLC__1SSV_20150507T015751_20150507T0158...,2015-05-07 01:57:51+00:00,2015-05-07 01:58:21+00:00,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,137,VV,5809,IW,2.652423,5,2015-05-07 00:00:00+00:00,"POLYGON ((-121.02775 34.52478, -120.61049 32.7..."
3,S1A_IW_SLC__1SSV_20150507T015818_20150507T0158...,2015-05-07 01:58:18+00:00,2015-05-07 01:58:45+00:00,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,137,VV,5809,IW,2.437214,5,2015-05-07 00:00:00+00:00,"POLYGON ((-121.37492 36.01932, -120.98943 34.3..."
4,S1A_IW_SLC__1SSV_20150507T015843_20150507T0159...,2015-05-07 01:58:43+00:00,2015-05-07 01:59:11+00:00,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,137,VV,5809,IW,2.568650,5,2015-05-07 00:00:00+00:00,"POLYGON ((-121.74913 37.56177, -121.34393 35.8..."


In [11]:
df_stack.repeat_pass_timestamp[0].tz

<UTC>

In [12]:
import datetime

x = pd.to_datetime(df_stack.repeat_pass_timestamp)
cond_0 = x > pd.Timestamp('2022-9-1', tz='UTC')
cond_1 = x < pd.Timestamp('2023-1-1', tz='UTC')

df_stack_subset = df_stack[cond_0 & cond_1].reset_index(drop=True)
df_stack_subset.shape

(30, 12)

In [13]:
df_stack_subset.to_file('sample_stack_137.geojson', driver='GeoJSON')

In [17]:
dates = df_stack_subset.repeat_pass_timestamp.unique().tolist()
dates

[Timestamp('2022-09-03 00:00:00+0000', tz='UTC'),
 Timestamp('2022-09-15 00:00:00+0000', tz='UTC'),
 Timestamp('2022-09-27 00:00:00+0000', tz='UTC'),
 Timestamp('2022-10-09 00:00:00+0000', tz='UTC'),
 Timestamp('2022-10-21 00:00:00+0000', tz='UTC'),
 Timestamp('2022-11-02 00:00:00+0000', tz='UTC'),
 Timestamp('2022-11-14 00:00:00+0000', tz='UTC'),
 Timestamp('2022-11-26 00:00:00+0000', tz='UTC'),
 Timestamp('2022-12-08 00:00:00+0000', tz='UTC'),
 Timestamp('2022-12-20 00:00:00+0000', tz='UTC')]

In [15]:
from s1_frame_enumerator.ifg_enum import select_ifg_pair_from_stack
frames = [S1Frame(21248), S1Frame(21249)]

ref_date = pd.Timestamp('2022-12-20', tz='UTC')
sec_date = pd.Timestamp('2022-12-8', tz='UTC')

select_ifg_pair_from_stack(ref_date, sec_date, df_stack_subset, None)

{'reference': ['S1A_IW_SLC__1SDV_20221220T015822_20221220T015852_046409_058F3D_D201',
  'S1A_IW_SLC__1SDV_20221220T015850_20221220T015918_046409_058F3D_2768',
  'S1A_IW_SLC__1SDV_20221220T015916_20221220T015943_046409_058F3D_8C7E'],
 'secondary': ['S1A_IW_SLC__1SDV_20221208T015823_20221208T015853_046234_058941_FEE4',
  'S1A_IW_SLC__1SDV_20221208T015851_20221208T015919_046234_058941_A19D',
  'S1A_IW_SLC__1SDV_20221208T015917_20221208T015944_046234_058941_52CD'],
 'reference_date': Timestamp('2022-12-20 00:00:00+0000', tz='UTC'),
 'secondary_date': Timestamp('2022-12-08 00:00:00+0000', tz='UTC'),
 'frame_id': None,
 'geometry': None}

In [16]:
from s1_frame_enumerator.ifg_enum import select_ifg_pair_from_stack
frames = [S1Frame(21248), S1Frame(21249)]

ref_date = pd.Timestamp('2022-12-20', tz='UTC')
sec_date = pd.Timestamp('2022-12-8', tz='UTC')

select_ifg_pair_from_stack(ref_date, sec_date, df_stack_subset, f)

NameError: name 'f' is not defined